In [ ]:
# Download the HMDB, and Lipidmaps database, and create pickle dictionaries for conversion.

In [ ]:
import zipfile as zf
import requests
import xml.etree.ElementTree as ET
import pickle

In [ ]:
# Save python dictionary in pickle
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
print ('Downloading HMDB database...')

u = requests.get("http://www.hmdb.ca/system/downloads/current/hmdb_metabolites.zip")
with open("hmdb_metabolites.zip", "wb") as f:
    f.write(u.content)

In [ ]:
# HMDB-KEGG convertion dictionary
print ('Found '+str(len(zf.ZipFile('hmdb_metabolites.zip').namelist()))+' entries')
print ('Finding equivalents...')

conv={}
with zf.ZipFile('hmdb_metabolites.zip') as myzip:
    for x in myzip.namelist():
        if x != 'hmdb_metabolites.xml':
            with myzip.open(x) as myfile:
                root = ET.fromstring(myfile.read())
                a = root.find('kegg_id').text
                b = root.find('accession').text
                if a != None:
                    conv[b] = a.rstrip()
                else:
                    continue
print ('Found '+str(len(conv))+' equivalents (HMDB-KEGG)')

In [ ]:
save_obj(conv, 'HMDB2KEGG')

In [ ]:
# Find LipidMaps dowload link
a = requests.get('http://lipidmaps.org/resources/downloads/index.html').text
b = a.splitlines()
for x in b:
    if '''.zip'>LMSDFDownload''' in x:
        r = x.split('''.zip'>''')[1].split('<')[0]

In [ ]:
# Download LipipMaps database
print ('')
print ('Downloading LipidMaps database...')

u = requests.get("http://lipidmaps.org/resources/downloads/"+r)
with open('LMSDF.zip', "wb") as f:
    f.write(u.content)

In [ ]:
# LipidMaps-KEGG convertion dictionary
d={}
with zf.ZipFile('LMSDF.zip') as myzip:
    for x in myzip.namelist():
        if x.endswith('All.sdf'):      
            with myzip.open(x) as myfile:
                a = bytes.decode(myfile.read())
                a = a.split('$$$$')
                print ('Found '+str(len(a))+' entries.')
                print ('Finding equivalents...')
                for x in a:
                    x = x.split('\n\n')
                    for y in x:
                        if y.startswith('> <KEGG_ID>'):
                            if x[0].split('\n')[1].startswith('  LIPDMAPS'): # Este if garante que o 1º elemento também é adicionado corretamente
                                d[x[0].split('\n')[0]] = y[len('> <KEGG_ID>\n'):]
                            else:
                                d[x[0].split('\n')[1]] = y[len('> <KEGG_ID>\n'):]
print ('Found '+str(len(d))+' equivalents (KEGG-LipidMaps)')

In [ ]:
d2 = {}
for key in d:
    d2[d[key]] = key

In [ ]:
save_obj(d, 'LM2KEGG')
save_obj(d2, 'KEGG2LM')